## Importations ##

In [1]:
import pandas as pd
import shapefile
import numpy as np
import matplotlib.pyplot as plt
from dbfread import DBF
from shapely.geometry import Polygon, Point
from time import time
from flask import Flask, request, abort, jsonify
from flask_login import LoginManager, current_user, login_user
from flask_cors import CORS
import request
import pickle

## useful functions  ##

In [2]:
def calories_per_ha_country(country):
    df = data[data['Country']==country]
    res = df['calories_per_ha'].mean()
    return res

In [3]:
def mean_calories_per_ha():
    res = {}
    for c in countries_all:
        res[c] = calories_per_ha_country(c)

    return res

In [4]:
def calories_per_country(country):
    df = data[data['Country'] == country]
    res = df['Calories'].mean()
    return res

In [5]:
def mean_calories_per_country():
    res = {}
    for c in countries_all:
        res[c] = calories_per_country(c)
        
    return res

In [6]:
def population_per_country(country):
    df = data[data['Country'] == country]
    res = df['population'].mean()
    return res

In [7]:
def mean_population_per_country():
    res = {}
    for c in countries_all:
        res[c] = population_per_country(c)
        
    return res

In [8]:
#Function that check the country for every point.
count = {}
def check_country(raw):
    cont = []
    p = raw['Point']
    id_ = raw['pixel_id']
    for i in dico:
        for j in dico[i]:
            if(j.contains(p)):
                #print(i, id_)
                count[id_]=i
            

In [146]:
mean_fr = calories_per_country('France')
mean_fr

19001479112.574627

In [35]:
df = data[data['Country'] == 'France']
df.head()

,pixel_id,log(cal_per_ha),calories_per_ha,Calories,%cropland,Change_in_Yield,Change_in_Production,log(cal_per_ha) 2000,calories_per_ha 2000,Calories 2000,...,precip,workability_index,slope,altitude,lat,lon,population,ha_per_cell_5m,Country,Point
808799,4573240,15.045196,3420152.0,1.040779e+10,0.0,-3.058107e+06,-7.044765e+09,15.683962,6478260.0,1.745255e+10,...,554,1.0,-0.264595,111.0,-1.75,43.333333,3194.0,8601.170,France,POINT (-1.75 43.333333)
809328,4577560,15.023600,3347082.0,1.635735e+10,1.0,-3.131177e+06,-1.524423e+10,15.683962,6478260.0,3.160158e+10,...,554,1.0,-0.269875,98.0,-1.67,43.333333,3064.0,8601.562,France,POINT (-1.67 43.333333)
809855,4581880,15.068539,3500928.0,2.751379e+10,1.0,-2.883740e+06,-2.180608e+10,15.669410,6384669.0,4.931987e+10,...,554,1.0,-0.321540,90.0,-1.58,43.333333,2620.0,8601.935,France,POINT (-1.58 43.333333)
809856,4581881,15.101600,3618607.0,3.083551e+10,1.0,-2.766062e+06,-2.379925e+10,15.669410,6384669.0,5.463476e+10,...,551,1.0,-0.359990,78.0,-1.58,43.416667,2340.0,8601.935,France,POINT (-1.58 43.416667)
810381,4586200,15.030915,3371658.0,2.809855e+10,1.0,-3.013010e+06,-2.436669e+10,15.669410,6384669.0,5.246524e+10,...,554,1.0,-0.331060,79.0,-1.50,43.333333,2199.0,8602.289,France,POINT (-1.5 43.333333)


In [147]:
def calories_per_ha_region(country):
    
    res = country.groupby('region').mean()['calories_per_ha']
    
    return res

## server part ##

In [148]:
app = Flask('J4U-Server')
CORS(app)

def ret_country():
    res_calcul = {
        'a': 1,
        'b': 2
    }
    return res_calcul

@app.route('/recom', methods=['GET'])
def recomend():
    res = ret_country()
    return jsonify(res)

@app.route('/mean_calo_ha', methods=['GET'])
def mean_calo_ha():
    res = mean_calories_per_ha()
    return jsonify(res)

@app.route('/mean_calo', methods =['GET'])
def mean_calo():
    res = mean_calories_per_country()
    return jsonify(res)

@app.route('/mean_pop', methods =['GET'])
def mean_pop():
    res = mean_population_per_country()
    return jsonify(res)

@app.route('/mean_calo_ha_region', methods = ['GET'])
def mean_calo_ha_reg():
    df = data[data['Country'] == 'France']
    res = calories_per_ha_region(df)
    return jsonify(res)


if __name__ == "__main__":
    app.run()

 * Serving Flask app "J4U-Server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2018-11-21 18:12:41,981] ERROR in app: Exception on /mean_calo_ha_region [GET]
Traceback (most recent call last):
  File "/home/nelsi11120/.local/lib/python3.7/site-packages/flask/app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/nelsi11120/.local/lib/python3.7/site-packages/flask/app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/nelsi11120/.local/lib/python3.7/site-packages/flask_cors/extension.py", line 161, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/home/nelsi11120/.local/lib/python3.7/site-packages/flask/app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/nelsi11120/.local/lib/python3.7/site-packages/flask/_compat.py", line 35, in reraise
    raise value
  File "/home/nelsi11120/.local/lib/python3.7/site-packages/flask/app.py", line 1813,

In [151]:
df = data[data['Country'] == 'France']
df.head()

,pixel_id,log(cal_per_ha),calories_per_ha,Calories,%cropland,Change_in_Yield,Change_in_Production,log(cal_per_ha) 2000,calories_per_ha 2000,Calories 2000,...,precip,workability_index,slope,altitude,lat,lon,population,ha_per_cell_5m,Country,Point
808799,4573240,15.045196,3420152.0,1.040779e+10,0.0,-3.058107e+06,-7.044765e+09,15.683962,6478260.0,1.745255e+10,...,554,1.0,-0.264595,111.0,-1.75,43.333333,3194.0,8601.170,France,POINT (-1.75 43.333333)
809328,4577560,15.023600,3347082.0,1.635735e+10,1.0,-3.131177e+06,-1.524423e+10,15.683962,6478260.0,3.160158e+10,...,554,1.0,-0.269875,98.0,-1.67,43.333333,3064.0,8601.562,France,POINT (-1.67 43.333333)
809855,4581880,15.068539,3500928.0,2.751379e+10,1.0,-2.883740e+06,-2.180608e+10,15.669410,6384669.0,4.931987e+10,...,554,1.0,-0.321540,90.0,-1.58,43.333333,2620.0,8601.935,France,POINT (-1.58 43.333333)
809856,4581881,15.101600,3618607.0,3.083551e+10,1.0,-2.766062e+06,-2.379925e+10,15.669410,6384669.0,5.463476e+10,...,551,1.0,-0.359990,78.0,-1.58,43.416667,2340.0,8601.935,France,POINT (-1.58 43.416667)
810381,4586200,15.030915,3371658.0,2.809855e+10,1.0,-3.013010e+06,-2.436669e+10,15.669410,6384669.0,5.246524e+10,...,554,1.0,-0.331060,79.0,-1.50,43.333333,2199.0,8602.289,France,POINT (-1.5 43.333333)


## data part ##

    loading data 

In [9]:
#results of the countries for every point, we stored it on a pickle file
#rez = pickle.load(open('ret.pkl','rb'))
#countries = [rez[i][1] for i in range(len(rez))]

In [10]:
rez1 = pickle.load(open('SSP/SSP1_cc.csv.pkl', 'rb'))
countries = [rez1[i][1] for i in range(len(rez1))]

In [12]:
#data provided by stanford
data = pd.read_csv('SSP1_cc.csv')

#set countries from pickle as 'Country' column for each pixel_id
data['Country'] = countries


#data from http://thematicmapping.org/downloads/world_borders.php in shapefile format
#in order to know in which country belongs the (lat,lon) points.
shape = shapefile.Reader("TM_WORLD_BORDERS-0.3.shp")


     creation of the pieces of countries (to locate every point in our dataset) 

In [13]:
#list of all countries in the shapefile file.
countries_all = []
for record in DBF("TM_WORLD_BORDERS-0.3.dbf"):
    countries_all.append(record['NAME'])

In [14]:
#in this part, we create a polygon with every points of the borders of every segments
#of every country (ie island, land etc..)
# in order to create a dictionnary with the name of countries in key and the 
# polygon corresponding to it. 

#initial variables.
dico = {}
co = Polygon()
final = []


#For every shape in the file
for shape in list(shape.iterShapes()):
    #initial variables within for loop.
    poly = []
    curr_count = []
    points_c = []
    poly = []
    npoints=len(shape.points) # total points
    nparts = len(shape.parts) # total parts

    #if country as only one part (no island, no discontinuities..)
    if nparts == 1:
        x_lon = np.zeros((len(shape.points),1))
        y_lat = np.zeros((len(shape.points),1))
        #for every point of the borders.
        for ip in range(len(shape.points)):
            x_lon[ip] = shape.points[ip][0]
            y_lat[ip] = shape.points[ip][1]
            points_c = (x_lon[ip][0],y_lat[ip][0])
            curr_count.append(points_c)

        
        #create a polygon with all the points of the border.
        co = Polygon(curr_count)
        #append the polygon to the list. 
        poly.append(co)

    #basically same thing as previous part but this time
    #we keep creating polygons for every segment of the country
    else: # loop over parts of each shape, plot separately
        for ip in range(nparts): # loop over parts, plot separately
            i0=shape.parts[ip]
            if ip < nparts-1:
                i1 = shape.parts[ip+1]-1
            else:
                i1 = npoints
            
            curr_count = []
            seg=shape.points[i0:i1+1]
            x_lon = np.zeros((len(seg),1))
            y_lat = np.zeros((len(seg),1))
            for ip in range(len(seg)):
                x_lon[ip] = seg[ip][0]
                y_lat[ip] = seg[ip][1]
                points_c = (x_lon[ip][0],y_lat[ip][0])
                curr_count.append(points_c)
   
            #create polygon
            co = Polygon(curr_count)
            #append to the list
            poly.append(co)
            
     
    #append all the polygons to the "final" list
    final.append(poly)

        



#make correspondance between countries
#and the list of all its polygons
for i in range(len(countries_all)):
    dico[countries_all[i]] = final[i]

    attribution of a country for each point of our dataset 

In [15]:
#due to the algo, we have to take the lon/lat (opposed to lat/lon) of every points in our dataset.
pix = data[['lat','lon']]
#we have to create a Point with the (lon/lat) of every pixel
data['Point'] = pix.apply(Point,axis=1)

In [17]:
#Apply the check country to every pixel_id
#l = r.apply(check_countryt,axis=1)

In [97]:
df_fr = data[data['Country'] == 'France']
df_fr.head()

,pixel_id,log(cal_per_ha),calories_per_ha,Calories,%cropland,Change_in_Yield,Change_in_Production,log(cal_per_ha) 2000,calories_per_ha 2000,Calories 2000,...,precip,workability_index,slope,altitude,lat,lon,population,ha_per_cell_5m,Country,Point
808799,4573240,15.045196,3420152.0,1.040779e+10,0.0,-3.058107e+06,-7.044765e+09,15.683962,6478260.0,1.745255e+10,...,554,1.0,-0.264595,111.0,-1.75,43.333333,3194.0,8601.170,France,POINT (-1.75 43.333333)
809328,4577560,15.023600,3347082.0,1.635735e+10,1.0,-3.131177e+06,-1.524423e+10,15.683962,6478260.0,3.160158e+10,...,554,1.0,-0.269875,98.0,-1.67,43.333333,3064.0,8601.562,France,POINT (-1.67 43.333333)
809855,4581880,15.068539,3500928.0,2.751379e+10,1.0,-2.883740e+06,-2.180608e+10,15.669410,6384669.0,4.931987e+10,...,554,1.0,-0.321540,90.0,-1.58,43.333333,2620.0,8601.935,France,POINT (-1.58 43.333333)
809856,4581881,15.101600,3618607.0,3.083551e+10,1.0,-2.766062e+06,-2.379925e+10,15.669410,6384669.0,5.463476e+10,...,551,1.0,-0.359990,78.0,-1.58,43.416667,2340.0,8601.935,France,POINT (-1.58 43.416667)
810381,4586200,15.030915,3371658.0,2.809855e+10,1.0,-3.013010e+06,-2.436669e+10,15.669410,6384669.0,5.246524e+10,...,554,1.0,-0.331060,79.0,-1.50,43.333333,2199.0,8602.289,France,POINT (-1.5 43.333333)


    attribution of a country for each point of our dataset 

In [36]:
regions = []
for record in DBF("FRA_adm/FRA_adm1.dbf",encoding = 'utf8'):
    regions.append(record['NAME_1'])
    #print(record)

In [52]:
#data from http://thematicmapping.org/downloads/world_borders.php in shapefile format
#in order to know in which country belongs the (lat,lon) points.
shape = shapefile.Reader("FRA_adm/FRA_adm1.dbf")


In [53]:
#in this part, we create a polygon with every points of the borders of every segments
#of every country (ie island, land etc..)
# in order to create a dictionnary with the name of countries in key and the 
# polygon corresponding to it. 

#initial variables.
dico_region_c = {}
co = Polygon()
final = []


#For every shape in the file
for shape in list(shape.iterShapes()):
    #initial variables within for loop.
    poly = []
    curr_count = []
    points_c = []
    poly = []
    npoints=len(shape.points) # total points
    nparts = len(shape.parts) # total parts

    #if country as only one part (no island, no discontinuities..)
    if nparts == 1:
        x_lon = np.zeros((len(shape.points),1))
        y_lat = np.zeros((len(shape.points),1))
        #for every point of the borders.
        for ip in range(len(shape.points)):
            x_lon[ip] = shape.points[ip][0]
            y_lat[ip] = shape.points[ip][1]
            points_c = (x_lon[ip][0],y_lat[ip][0])
            curr_count.append(points_c)

        
        #create a polygon with all the points of the border.
        co = Polygon(curr_count)
        #append the polygon to the list. 
        poly.append(co)

    #basically same thing as previous part but this time
    #we keep creating polygons for every segment of the country
    else: # loop over parts of each shape, plot separately
        for ip in range(nparts): # loop over parts, plot separately
            i0=shape.parts[ip]
            if ip < nparts-1:
                i1 = shape.parts[ip+1]-1
            else:
                i1 = npoints
            
            curr_count = []
            seg=shape.points[i0:i1+1]
            x_lon = np.zeros((len(seg),1))
            y_lat = np.zeros((len(seg),1))
            for ip in range(len(seg)):
                x_lon[ip] = seg[ip][0]
                y_lat[ip] = seg[ip][1]
                points_c = (x_lon[ip][0],y_lat[ip][0])
                curr_count.append(points_c)
   
            #create polygon
            co = Polygon(curr_count)
            #append to the list
            poly.append(co)
            
     
    #append all the polygons to the "final" list
    final.append(poly)

        



#make correspondance between countries
#and the list of all its polygons
for i in range(len(regions)):
    dico_region_c[regions[i]] = final[i]

In [55]:
dico_region = {}
dico_region['France'] = dico_region_c

In [117]:
#Function that check the country for every point.
count = {}
def check_region(raw):
    cont = []
    p = raw['Point']
    id_ = raw['pixel_id']
    cc = raw['Country']
    found = 0
    for i in dico_region[cc]:
        for j in dico_region[cc][i]:
            if(j.contains(p)):
                #print(i, id_)
                count[id_]=i
                found = 1
        if(found == 0):
            count[id_] = None
            
                

In [118]:
ad = df_fr.apply(check_region,axis=1)

In [127]:
df_fr['Region'] = count.values()

/home/nelsi11120/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [128]:
df_fr

,pixel_id,log(cal_per_ha),calories_per_ha,Calories,%cropland,Change_in_Yield,Change_in_Production,log(cal_per_ha) 2000,calories_per_ha 2000,Calories 2000,...,workability_index,slope,altitude,lat,lon,population,ha_per_cell_5m,Country,Point,region
808799,4573240,15.045196,3420152.0,1.040779e+10,0.0,-3.058107e+06,-7.044765e+09,15.683962,6478260.0,1.745255e+10,...,1.0,-0.264595,111.0,-1.75,43.333333,3194.0,8601.170,France,POINT (-1.75 43.333333),Aquitaine
809328,4577560,15.023600,3347082.0,1.635735e+10,1.0,-3.131177e+06,-1.524423e+10,15.683962,6478260.0,3.160158e+10,...,1.0,-0.269875,98.0,-1.67,43.333333,3064.0,8601.562,France,POINT (-1.67 43.333333),Aquitaine
809855,4581880,15.068539,3500928.0,2.751379e+10,1.0,-2.883740e+06,-2.180608e+10,15.669410,6384669.0,4.931987e+10,...,1.0,-0.321540,90.0,-1.58,43.333333,2620.0,8601.935,France,POINT (-1.58 43.333333),Aquitaine
809856,4581881,15.101600,3618607.0,3.083551e+10,1.0,-2.766062e+06,-2.379925e+10,15.669410,6384669.0,5.463476e+10,...,1.0,-0.359990,78.0,-1.58,43.416667,2340.0,8601.935,France,POINT (-1.58 43.416667),Aquitaine
810381,4586200,15.030915,3371658.0,2.809855e+10,1.0,-3.013010e+06,-2.436669e+10,15.669410,6384669.0,5.246524e+10,...,1.0,-0.331060,79.0,-1.50,43.333333,2199.0,8602.289,France,POINT (-1.5 43.333333),Aquitaine
810382,4586201,15.032797,3378008.0,2.890741e+10,1.0,7.288999e+05,6.711848e+09,14.789734,2649108.0,2.219557e+10,...,1.0,-0.369125,69.0,-1.50,43.416667,2151.0,8602.289,France,POINT (-1.5 43.416667),Aquitaine
810383,4586202,15.028898,3364864.0,2.867445e+10,1.0,7.157560e+05,6.052050e+09,14.789734,2649108.0,2.262240e+10,...,1.0,-0.477455,61.0,-1.50,43.500000,2151.0,8602.289,France,POINT (-1.5 43.5),Aquitaine
810933,4590517,14.968582,3167909.0,4.764195e+09,0.0,-3.216760e+06,-4.837663e+09,15.669410,6384669.0,9.601858e+09,...,1.0,-0.359320,75.0,-1.42,43.083333,2465.0,8602.626,France,POINT (-1.42 43.083333),Aquitaine
810936,4590520,15.032669,3377577.0,2.687909e+10,1.0,3.377576e+06,2.687909e+10,0.000000,1.0,8.164000e+03,...,1.0,-0.363365,66.0,-1.42,43.333333,2161.0,8602.626,France,POINT (-1.42 43.333333),Aquitaine
810937,4590521,15.075472,3525285.0,2.821236e+10,1.0,-2.859383e+06,-2.362624e+10,15.669410,6384669.0,5.183860e+10,...,1.0,-0.433044,59.0,-1.42,43.416667,2151.0,8602.626,France,POINT (-1.42 43.416667),Aquitaine


In [140]:
a = calories_per_ha_region(df_fr)

In [143]:
a

region
Aquitaine        2.716714e+06
Midi-Pyrénées    2.487494e+06
Name: calories_per_ha, dtype: float64

In [136]:
df_fr.groupby('region').mean()#['calories_per_ha']

,pixel_id,log(cal_per_ha),calories_per_ha,Calories,%cropland,Change_in_Yield,Change_in_Production,log(cal_per_ha) 2000,calories_per_ha 2000,Calories 2000,%cropland 2000,temp_avg,precip,workability_index,slope,altitude,lat,lon,population,ha_per_cell_5m
region,,,,,,,,,,,,,,,,,,,,
Aquitaine,4.612191e+06,14.798113,2.716714e+06,1.917105e+10,0.934959,1.950401e+06,1.378662e+10,2.252283,7.663130e+05,5.384429e+09,0.96748,277.447154,533.764228,1.528455,-1.708567,42.95935,-0.998618,43.364499,5842.739837,8603.898455
Midi-Pyrénées,4.655854e+06,14.724642,2.487494e+06,1.784041e+10,1.000000,4.655762e+05,1.373210e+09,5.787130,2.021918e+06,1.646720e+10,1.00000,277.375000,537.500000,1.125000,-0.752674,27.62500,-0.156250,42.854167,12994.750000,8605.489625


In [109]:
vaud = Point(-1.33,43.08)

In [110]:
for i in dico:
    for j in dico[i]:
           # print('a',j)
        if(j.contains(vaud)):
            print(i)
               

In [66]:
dico

{'Alsace': [<shapely.geometry.polygon.Polygon at 0x7f839a2a1f98>],
 'Aquitaine': [<shapely.geometry.polygon.Polygon at 0x7f839a2a1e48>,
 'Auvergne': [<shapely.geometry.polygon.Polygon at 0x7f83a1ca0c88>],
 'Île-de-France': [<shapely.geometry.polygon.Polygon at 0x7f83a1ca0c18>],
 'Basse-Normandie': [<shapely.geometry.polygon.Polygon at 0x7f83a1ca0a20>,
 'Bourgogne': [<shapely.geometry.polygon.Polygon at 0x7f83a1996a90>,
 'Bretagne': [<shapely.geometry.polygon.Polygon at 0x7f83a1996c88>,
 'Centre': [<shapely.geometry.polygon.Polygon at 0x7f838e6ed470>],
 'Champagne-Ardenne': [<shapely.geometry.polygon.Polygon at 0x7f838e6ed4a8>],
 'Corse': [<shapely.geometry.polygon.Polygon at 0x7f838e6ed4e0>,
 'Franche-Comté': [<shapely.geometry.polygon.Polygon at 0x7f838e6ed9e8>,
 'Haute-Normandie': [<shapely.geometry.polygon.Polygon at 0x7f838e6eda58>,
 'Languedoc-Roussillon': [<shapely.geometry.polygon.Polygon at 0x7f838e6edac8>,
 'Limousin': [<shapely.geometry.polygon.Polygon at 0x7f838e6edc88>],
 '

In [65]:
dico_region['France']

{'Alsace': [<shapely.geometry.polygon.Polygon at 0x7f838e6fdb70>],
 'Aquitaine': [<shapely.geometry.polygon.Polygon at 0x7f838e80cbe0>,
 'Auvergne': [<shapely.geometry.polygon.Polygon at 0x7f838e80cda0>],
 'Île-de-France': [<shapely.geometry.polygon.Polygon at 0x7f838e80cd68>],
 'Basse-Normandie': [<shapely.geometry.polygon.Polygon at 0x7f838e80ccc0>,
 'Bourgogne': [<shapely.geometry.polygon.Polygon at 0x7f838ec4a160>,
 'Bretagne': [<shapely.geometry.polygon.Polygon at 0x7f838ec4a1d0>,
 'Centre': [<shapely.geometry.polygon.Polygon at 0x7f8399315828>],
 'Champagne-Ardenne': [<shapely.geometry.polygon.Polygon at 0x7f8399315860>],
 'Corse': [<shapely.geometry.polygon.Polygon at 0x7f8399315898>,
 'Franche-Comté': [<shapely.geometry.polygon.Polygon at 0x7f8399315da0>,
 'Haute-Normandie': [<shapely.geometry.polygon.Polygon at 0x7f8399315e10>,
 'Languedoc-Roussillon': [<shapely.geometry.polygon.Polygon at 0x7f8399315e80>,
 'Limousin': [<shapely.geometry.polygon.Polygon at 0x7f83994ef080>],
 '

In [42]:
df.head()

,pixel_id,log(cal_per_ha),calories_per_ha,Calories,%cropland,Change_in_Yield,Change_in_Production,log(cal_per_ha) 2000,calories_per_ha 2000,Calories 2000,...,precip,workability_index,slope,altitude,lat,lon,population,ha_per_cell_5m,Country,Point
808799,4573240,15.045196,3420152.0,1.040779e+10,0.0,-3.058107e+06,-7.044765e+09,15.683962,6478260.0,1.745255e+10,...,554,1.0,-0.264595,111.0,-1.75,43.333333,3194.0,8601.170,France,POINT (-1.75 43.333333)
809328,4577560,15.023600,3347082.0,1.635735e+10,1.0,-3.131177e+06,-1.524423e+10,15.683962,6478260.0,3.160158e+10,...,554,1.0,-0.269875,98.0,-1.67,43.333333,3064.0,8601.562,France,POINT (-1.67 43.333333)
809855,4581880,15.068539,3500928.0,2.751379e+10,1.0,-2.883740e+06,-2.180608e+10,15.669410,6384669.0,4.931987e+10,...,554,1.0,-0.321540,90.0,-1.58,43.333333,2620.0,8601.935,France,POINT (-1.58 43.333333)
809856,4581881,15.101600,3618607.0,3.083551e+10,1.0,-2.766062e+06,-2.379925e+10,15.669410,6384669.0,5.463476e+10,...,551,1.0,-0.359990,78.0,-1.58,43.416667,2340.0,8601.935,France,POINT (-1.58 43.416667)
810381,4586200,15.030915,3371658.0,2.809855e+10,1.0,-3.013010e+06,-2.436669e+10,15.669410,6384669.0,5.246524e+10,...,554,1.0,-0.331060,79.0,-1.50,43.333333,2199.0,8602.289,France,POINT (-1.5 43.333333)


#### test part ####

In [19]:

arara = Point(-117.682988,41.265722)

In [20]:
for i in dico:
    for j in dico[i]:
           # print('a',j)
        if(j.contains(arara)):
            print(i)
               

United States


In [19]:
data.head(1)['Calories 2000'] / data.head(1)['ha_per_cell_5m']

0    0.005417
dtype: float64

In [21]:
data.head(1)['calories_per_ha 2000']

0    1.0
Name: calories_per_ha 2000, dtype: float64